<a href="https://colab.research.google.com/github/VasquezSRE/machinelearning-gtc/blob/main/extraction/Feature%20Selection/3_SVM_FwrdSln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys; sys.path.append('./')
from utils import Utils
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
import time
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import qgrid

In [ ]:
ut = Utils()
X, Y = ut.get_matrixes()

In [ ]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [ ]:
def select_features(modelo, n_features, fwd, fltg):
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=2,
           scoring='accuracy',
           cv=0,
           n_jobs=-1,
           pre_dispatch=10)
    
    return sfs

In [ ]:
def trainSFS(n_features,clf):
    
    forward = True
    floating = False
   
        
        
    tiempo_i = time.time()
    
    #Implemetamos la metodología de validación 

    Folds = 4
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, Y, random_state=0, stratify=Y, shuffle=True)
    for train, test in skf.split(X_train_d,y_train_d):
        

        X_train, X_test = X[train], X[train]
        y_train, y_test = Y[train], Y[train]  
        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        sf = select_features(clf, n_features, forward, floating)

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        print("iteracion: " + str(j))
        print("feature: " + str(n_features))
        sf = sf.fit(X_train, y_train)

        Errores[j] = 1-sf.k_score_
        j+=1
    return  str(np.mean(Errores)), str(np.std(Errores)), str(sf.k_score_*100), str(time.time()-tiempo_i),sf.k_feature_idx_

In [ ]:
def train(clf):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Implemetamos la metodología de validación

    j = 0
    Folds = 4
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)

    for train, test in skf.split(X,Y):

        Xtrain, Ytrain, Xtest, Ytest = ut.get_training_test(X=X, Y=Y, train=train, test=test)

        #Aquí se entran y se valida el modelo sin hacer selección de características

        ######

        # Entrenamiento el modelo.
        model = clf.fit(Xtrain,Ytrain)

        # Validación del modelo
        ypred = model.predict(Xtest)

        #######

        Errores[j] = classification_error(ypred, Ytest)
        j+=1
    
    m_errores = np.mean(Errores)
    ic_errores = np.std(Errores)
    eficiencia = (1 - np.mean(Errores))*100
    tEjec = time.time()-tiempo_i
                        
    print('finalizado clf sin selección',str(m_errores), str(ic_errores), str(eficiencia), str(tEjec))
    return  m_errores, str(np.std(Errores)), str((1 - np.mean(Errores))*100), str(time.time()-tiempo_i)

In [ ]:
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin Selección','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SFS']),
    '# de características seleccionadas' : pd.Series([23, 3, 5, 10,15,20]),
   })
ic_ = []
tEjec_ = []
errorVal_ = []
eficiencia_ = []
caracteristicas=[]
model = OneVsRestClassifier(SVC(C=100,kernel='rbf', gamma=0.10, probability=True), n_jobs=-1)
for e, i in zip(df_types['Tecnica'], df_types['# de características seleccionadas']):
    
    if(e == "SVM sin Selección"):
        errorVal, ic, eficiencia, tEjec = train(model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
    else:
        print('entrenando: ',e,i)
        errorVal, ic, eficiencia, tEjec,car = trainSFS(i, model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
        caracteristicas.append(car)



finalizado clf sin selección 0.08559713674365252 0.02494185475035955 91.44028632563474 6.861554384231567
entrenando:  SVM + SFS 3
iteracion: 0
feature: 3


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   14.3s finished

[2021-01-20 01:26:26] Features: 1/3 -- score: 0.7754297994269341[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    9.8s finished

[2021-01-20 01:26:36] Features: 2/3 -- score: 0.8481375358166189[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    7.5s finished

[2021-01-20 01:26:43] Features: 3/3 -- score: 0.8825214899713467[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


iteracion: 1
feature: 3


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   14.4s finished

[2021-01-20 01:26:58] Features: 1/3 -- score: 0.75[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   10.8s finished

[2021-01-20 01:27:08] Features: 2/3 -- score: 0.8137535816618912[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    8.8s finished

[2021-01-20 01:27:17] Features: 3/3 -- score: 0.839541547277937[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


iteracion: 2
feature: 3


...

[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   14.3s finished

[2021-01-20 01:44:28] Features: 1/20 -- score: 0.75[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   10.8s finished

[2021-01-20 01:44:38] Features: 2/20 -- score: 0.8137535816618912[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    8.8s finished

[2021-01-20 01:44:47] Features: 3/20 -- score: 0.839541547277937[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    7.9s finished

[2021-01-20 01:44:55] Features: 4/20 -- score: 0.8674785100286533[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    6.6s finished

[2021-01-20 01:45:02] Features: 5/20 -- score: 0.8904011461318052[Parallel(n_jobs=-1)]: Usi

iteracion: 2
feature: 20


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   15.2s finished

[2021-01-20 01:46:24] Features: 1/20 -- score: 0.744269340974212[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   11.2s finished

[2021-01-20 01:46:35] Features: 2/20 -- score: 0.8166189111747851[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    8.4s finished

[2021-01-20 01:46:43] Features: 3/20 -- score: 0.8599570200573066[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    7.3s finished

[2021-01-20 01:46:51] Features: 4/20 -- score: 0.8878939828080229[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    6.3s finished

[2021-01-20 01:46:57] Features: 5/20 -- score: 0.9079512893982808[Parallel(n_

iteracion: 3
feature: 20


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   12.6s finished

[2021-01-20 01:48:08] Features: 1/20 -- score: 0.7769423558897243[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    9.9s finished

[2021-01-20 01:48:17] Features: 2/20 -- score: 0.8070175438596491[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    8.1s finished

[2021-01-20 01:48:26] Features: 3/20 -- score: 0.8539205155746509[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    7.1s finished

[2021-01-20 01:48:33] Features: 4/20 -- score: 0.8800572860723237[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    6.5s finished

[2021-01-20 01:48:39] Features: 5/20 -- score: 0.8933046902971715[Parallel(n

In [ ]:
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin Selección','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SFS']),
    '# de características seleccionadas' : pd.Series([23, 3, 5, 10,15,20]),
   })
df_types["Eficiencia(%)"] = eficiencia_
df_types["Error de validación"] = errorVal_
df_types["IC(std)"] = ic_
df_types["Tiempo de ejecución"] = tEjec_

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)
df_types.to_csv('car_select_SVM.csv')
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

/home/juan/simulation/simulation/lib/python3.8/site-packages/qgrid/grid.py:1329: FutureWarning: inplace is deprecated and will be removed in a future version.
  df.index.set_levels(col_series, level=key_index, inplace=True)


Eficiencia(%)  \
Tecnica           # de características seleccionadas                      
SVM sin Selección 23                                  91.44028632563474   
SVM + SFS         3                                   85.39205155746508   
                  5                                   89.33046902971715   
                  10                                  90.47619047619048   
                  15                                  91.83673469387756   
                  20                                  92.30218403150734   

                                                      Error de validación  \
Tecnica           # de características seleccionadas                        
SVM sin Selección 23                                             0.085597   
SVM + SFS         3                                   0.14101485677968972   
                  5                                   0.09965018845722573   
                  10                                  0.08228015418201665   
                  15                                  0.06786518332261268   
                  20                                  0.06571660295848095   

                                                                   IC(std)  \
Tecnica           # de características seleccionadas                         
SVM sin Selección 23                                   0.02494185475035955   
SVM + SFS         3                                   0.015480697976276518   
                  5                                    0.00858206861757379   
                  10                                  0.012095537938961086   
                  15                                  0.013452370285591335   
                  20                                  0.014037367360680494   

                                                       Tiempo de ejecución  
Tecnica           # de características seleccionadas                        
SVM sin Selección 23                                   6.862213611602783 s  
SVM + SFS         3                                   130.48278832435608 s  
                  5                                   185.89935421943665 s  
                  10                                    290.449494600296 s  
                  15                                   376.4632177352905 s  
                  20                                  428.12613463401794 s

In [ ]:
model = OneVsRestClassifier(SVC(C=100,kernel='rbf', gamma=0.10, probability=True), n_jobs=-1)

In [ ]:
_, _, _, _,car = trainSFS(20, model)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


iteracion: 0
feature: 20


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   14.7s finished

[2021-01-20 01:49:58] Features: 1/20 -- score: 0.7754297994269341[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   10.0s finished

[2021-01-20 01:50:08] Features: 2/20 -- score: 0.8481375358166189[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    7.8s finished

[2021-01-20 01:50:16] Features: 3/20 -- score: 0.8825214899713467[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    6.5s finished

[2021-01-20 01:50:22] Features: 4/20 -- score: 0.8954154727793696[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    6.2s finished

[2021-01-20 01:50:29] Features: 5/20 -- score: 0.9097421203438395[Parallel(n

iteracion: 1
feature: 20


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   14.4s finished

[2021-01-20 01:51:36] Features: 1/20 -- score: 0.75[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   10.9s finished

[2021-01-20 01:51:47] Features: 2/20 -- score: 0.8137535816618912[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    8.9s finished

[2021-01-20 01:51:56] Features: 3/20 -- score: 0.839541547277937[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    7.8s finished

[2021-01-20 01:52:04] Features: 4/20 -- score: 0.8674785100286533[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    6.7s finished

[2021-01-20 01:52:11] Features: 5/20 -- score: 0.8904011461318052[Parallel(n_jobs=-1)]: Usi

iteracion: 2
feature: 20


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   15.4s finished

[2021-01-20 01:53:33] Features: 1/20 -- score: 0.744269340974212[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   11.1s finished

[2021-01-20 01:53:44] Features: 2/20 -- score: 0.8166189111747851[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    8.5s finished

[2021-01-20 01:53:52] Features: 3/20 -- score: 0.8599570200573066[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    7.4s finished

[2021-01-20 01:54:00] Features: 4/20 -- score: 0.8878939828080229[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    6.3s finished

[2021-01-20 01:54:06] Features: 5/20 -- score: 0.9079512893982808[Parallel(n_

iteracion: 3
feature: 20


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   12.5s finished

[2021-01-20 01:55:16] Features: 1/20 -- score: 0.7769423558897243[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   10.0s finished

[2021-01-20 01:55:26] Features: 2/20 -- score: 0.8070175438596491[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    8.1s finished

[2021-01-20 01:55:34] Features: 3/20 -- score: 0.8539205155746509[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    7.1s finished

[2021-01-20 01:55:41] Features: 4/20 -- score: 0.8800572860723237[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    6.5s finished

[2021-01-20 01:55:48] Features: 5/20 -- score: 0.8933046902971715[Parallel(n

In [ ]:
car

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 19, 20, 21, 22)